In [ ]:
"""
Improved Deepfake Detection: CNN vs TCN Comparison
Fixed temporal modeling and robust training
"""

import subprocess
import sys
import os

def install(package):
    """Auto-install missing packages"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✓ Installed {package}")
    except:
        print(f"! Warning: Could not install {package}, continuing...")

# Auto-install required packages
required_packages = {
    'torch': 'torch torchvision',
    'cv2': 'opencv-python',
    'numpy': 'numpy',
    'sklearn': 'scikit-learn',
    'PIL': 'Pillow',
    'tqdm': 'tqdm',
    'matplotlib': 'matplotlib'
}

print("=== Checking and installing dependencies ===")
for module, package in required_packages.items():
    try:
        __import__(module)
    except ImportError:
        print(f"Installing {package}...")
        install(package)

# Now import everything
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import random
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n=== Using device: {DEVICE} ===")

# Paths
DATA_PATHS = {
    'Original': r"C:\Users\chakr\Downloads\archive (3)\FaceForensics++_C23\original",
    'NeuralTextures': r"C:\Users\chakr\Downloads\archive (3)\FaceForensics++_C23\NeuralTextures",
    'FaceSwap': r"C:\Users\chakr\Downloads\archive (3)\FaceForensics++_C23\FaceSwap",
    'FaceShifter': r"C:\Users\chakr\Downloads\archive (3)\FaceForensics++_C23\FaceShifter",
    'Face2Face': r"C:\Users\chakr\Downloads\archive (3)\FaceForensics++_C23\Face2Face",
    'Deepfakes': r"C:\Users\chakr\Downloads\archive (3)\FaceForensics++_C23\Deepfakes",
    'DeepFakeDetection': r"C:\Users\chakr\Downloads\archive (3)\FaceForensics++_C23\DeepFakeDetection"
}

# Improved Hyperparameters
IMG_SIZE = 128
SEQUENCE_LENGTH = 16  # More frames for better temporal patterns
BATCH_SIZE = 8  # Increased for stability
EPOCHS = 25
LEARNING_RATE = 0.0001  # Lower initial learning rate
MAX_VIDEOS_PER_CLASS = 50

class VideoDataset(Dataset):
    """Enhanced dataset with data augmentation"""
    def __init__(self, data_paths, sequence_length=16, img_size=128, max_videos=50, augment=False):
        self.sequence_length = sequence_length
        self.img_size = img_size
        self.augment = augment
        self.data = []
        
        print("\n=== Loading dataset ===")
        
        # Load real videos (label = 0)
        real_path = Path(data_paths['Original'])
        if real_path.exists():
            real_videos = list(real_path.glob('**/*.mp4'))[:max_videos]
            self.data.extend([(str(v), 0) for v in real_videos])
            print(f"Loaded {len(real_videos)} real videos")
        
        # Load fake videos (label = 1)
        fake_count = 0
        for key in ['NeuralTextures', 'FaceSwap', 'Deepfakes', 'Face2Face', 'FaceShifter']:
            if key in data_paths:
                fake_path = Path(data_paths[key])
                if fake_path.exists():
                    fake_videos = list(fake_path.glob('**/*.mp4'))[:max_videos//5]
                    self.data.extend([(str(v), 1) for v in fake_videos])
                    fake_count += len(fake_videos)
                    print(f"Loaded {len(list(fake_path.glob('**/*.mp4'))[:max_videos//5])} {key} videos")
        
        print(f"Total fake videos: {fake_count}")
        print(f"Total dataset size: {len(self.data)} videos")
        
        # Shuffle data
        random.shuffle(self.data)
        
        if len(self.data) == 0:
            raise ValueError("No videos found! Check your paths.")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        video_path, label = self.data[idx]
        
        try:
            frames = self.extract_frames(video_path)
            if self.augment:
                frames = self.augment_frames(frames)
            return frames, label
        except Exception as e:
            print(f"! Error loading {video_path}: {e}")
            # Return a valid tensor instead of random
            frames = torch.zeros(self.sequence_length, 3, self.img_size, self.img_size)
            return frames, label
    
    def extract_frames(self, video_path):
        """Extract frames from video with better error handling"""
        cap = cv2.VideoCapture(video_path)
        frames = []
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total_frames == 0:
            cap.release()
            raise ValueError(f"No frames in video: {video_path}")
        
        # Sample frames evenly
        if total_frames < self.sequence_length:
            indices = list(range(total_frames))
            while len(indices) < self.sequence_length:
                indices.extend(list(range(total_frames)))
            indices = indices[:self.sequence_length]
        else:
            indices = np.linspace(0, total_frames-1, self.sequence_length, dtype=int)
        
        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            
            if ret:
                frame = cv2.resize(frame, (self.img_size, self.img_size))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = frame.astype(np.float32) / 255.0
                # Normalize
                frame = (frame - 0.5) / 0.5
                frame = torch.from_numpy(frame).permute(2, 0, 1)
                frames.append(frame)
            else:
                if frames:
                    frames.append(frames[-1].clone())
                else:
                    frames.append(torch.zeros(3, self.img_size, self.img_size))
        
        cap.release()
        return torch.stack(frames)
    
    def augment_frames(self, frames):
        """Apply data augmentation"""
        # Random horizontal flip
        if random.random() > 0.5:
            frames = torch.flip(frames, dims=[3])
        
        # Random brightness adjustment
        if random.random() > 0.5:
            brightness_factor = random.uniform(0.8, 1.2)
            frames = frames * brightness_factor
            frames = torch.clamp(frames, -1, 1)
        
        return frames


class ImprovedCNN(nn.Module):
    """Improved CNN with better architecture"""
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 2
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 3
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Global pooling
            nn.AdaptiveAvgPool2d(1)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 2)
        )
    
    def forward(self, x):
        # x shape: [batch, seq_len, channels, height, width]
        batch_size, seq_len = x.shape[:2]
        
        # Process all frames
        x = x.view(batch_size * seq_len, *x.shape[2:])
        x = self.features(x)
        x = x.view(batch_size, seq_len, -1)
        
        # Temporal pooling (max and average)
        x_max = x.max(dim=1)[0]
        x_avg = x.mean(dim=1)
        x = x_max + x_avg  # Combine both
        
        x = self.classifier(x)
        return x


class TemporalBlock(nn.Module):
    """Fixed Temporal Convolutional Block"""
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super(TemporalBlock, self).__init__()
        
        # Use causal padding to preserve sequence length
        self.padding = (kernel_size - 1) * dilation
        
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                              padding=self.padding, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                              padding=self.padding, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        
        # Residual connection
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Apply convolutions
        out = self.conv1(x)
        if self.padding > 0:
            out = out[:, :, :-self.padding]  # Remove future padding
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out = self.conv2(out)
        if self.padding > 0:
            out = out[:, :, :-self.padding]
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
        # Residual connection
        res = x if self.downsample is None else self.downsample(x)
        
        # Match dimensions
        if out.size(2) != res.size(2):
            res = res[:, :, :out.size(2)]
        
        return self.relu(out + res)


class ImprovedTCN(nn.Module):
    """Improved TCN model with fixed temporal convolutions"""
    def __init__(self):
        super(ImprovedTCN, self).__init__()
        
        # Spatial feature extractor
        self.spatial = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 2
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 3
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.AdaptiveAvgPool2d(1)
        )
        
        # Temporal feature extractor (TCN)
        self.temporal = nn.Sequential(
            TemporalBlock(256, 256, kernel_size=3, dilation=1),
            TemporalBlock(256, 256, kernel_size=3, dilation=2),
            TemporalBlock(256, 256, kernel_size=3, dilation=4),
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 2)
        )
    
    def forward(self, x):
        # x shape: [batch, seq_len, channels, height, width]
        batch_size, seq_len = x.shape[:2]
        
        # Extract spatial features
        x = x.view(batch_size * seq_len, *x.shape[2:])
        x = self.spatial(x)
        x = x.view(batch_size, 256, seq_len)
        
        # Extract temporal features
        x = self.temporal(x)
        
        # Global temporal pooling (max + average)
        x_max = torch.max(x, dim=2)[0]
        x_avg = torch.mean(x, dim=2)
        x = x_max + x_avg
        
        # Classification
        x = self.classifier(x)
        return x


def train_model(model, train_loader, val_loader, model_name, epochs=25):
    """Training function with learning rate scheduling"""
    print(f"\n{'='*50}")
    print(f"Training {model_name}")
    print(f"{'='*50}")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', 
                                                      factor=0.5, patience=3)
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_preds, train_labels = [], []
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for batch_idx, (frames, labels) in enumerate(pbar):
            try:
                frames, labels = frames.to(DEVICE), labels.to(DEVICE)
                
                optimizer.zero_grad()
                outputs = model(frames)
                loss = criterion(outputs, labels)
                loss.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                
                train_loss += loss.item()
                train_preds.extend(outputs.argmax(dim=1).cpu().numpy())
                train_labels.extend(labels.cpu().numpy())
                
                pbar.set_postfix({'loss': f'{loss.item():.4f}'})
            except Exception as e:
                print(f"! Error in batch {batch_idx}: {e}")
                continue
        
        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, train_preds)
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds, val_labels = [], []
        
        with torch.no_grad():
            for frames, labels in val_loader:
                try:
                    frames, labels = frames.to(DEVICE), labels.to(DEVICE)
                    outputs = model(frames)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item()
                    val_preds.extend(outputs.argmax(dim=1).cpu().numpy())
                    val_labels.extend(labels.cpu().numpy())
                except Exception as e:
                    print(f"! Validation error: {e}")
                    continue
        
        val_loss /= len(val_loader)
        val_acc = accuracy_score(val_labels, val_preds)
        
        # Learning rate scheduling
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_acc)
        new_lr = optimizer.param_groups[0]['lr']
        if new_lr != old_lr:
            print(f"✓ Learning rate reduced from {old_lr:.6f} to {new_lr:.6f}")
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f'{model_name}_best.pth')
            print(f"✓ Saved best model with Val Acc: {val_acc:.4f}")
    
    # Load best model
    model.load_state_dict(torch.load(f'{model_name}_best.pth'))
    return history


def evaluate_model(model, test_loader, model_name):
    """Evaluate model performance"""
    model.eval()
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for frames, labels in tqdm(test_loader, desc=f"Evaluating {model_name}"):
            try:
                frames = frames.to(DEVICE)
                outputs = model(frames)
                all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
                all_labels.extend(labels.numpy())
            except:
                continue
    
    acc = accuracy_score(all_labels, all_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    cm = confusion_matrix(all_labels, all_preds)
    
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1}


def plot_results(cnn_history, tcn_history, cnn_results, tcn_results):
    """Plot comparison results"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Training Loss
    axes[0, 0].plot(cnn_history['train_loss'], label='CNN Train', marker='o', linewidth=2)
    axes[0, 0].plot(tcn_history['train_loss'], label='TCN Train', marker='s', linewidth=2)
    axes[0, 0].set_title('Training Loss Comparison', fontsize=14, weight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Validation Accuracy
    axes[0, 1].plot(cnn_history['val_acc'], label='CNN Val', marker='o', linewidth=2)
    axes[0, 1].plot(tcn_history['val_acc'], label='TCN Val', marker='s', linewidth=2)
    axes[0, 1].set_title('Validation Accuracy Comparison', fontsize=14, weight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Final Metrics Comparison
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    cnn_vals = [cnn_results[m] for m in metrics]
    tcn_vals = [tcn_results[m] for m in metrics]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    bars1 = axes[1, 0].bar(x - width/2, cnn_vals, width, label='CNN', alpha=0.8, color='#2196F3')
    bars2 = axes[1, 0].bar(x + width/2, tcn_vals, width, label='TCN', alpha=0.8, color='#4CAF50')
    axes[1, 0].set_title('Final Performance Metrics', fontsize=14, weight='bold')
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels([m.capitalize() for m in metrics])
    axes[1, 0].legend()
    axes[1, 0].grid(True, axis='y', alpha=0.3)
    axes[1, 0].set_ylim([0, 1])
    
    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                          f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    # Improvement Table
    axes[1, 1].axis('off')
    improvements = [(m, tcn_vals[i] - cnn_vals[i], 
                    (tcn_vals[i] - cnn_vals[i])/cnn_vals[i]*100 if cnn_vals[i] > 0 else 0) 
                   for i, m in enumerate(metrics)]
    
    table_data = [['Metric', 'CNN', 'TCN', 'Improvement']]
    for i, m in enumerate(metrics):
        improvement_str = f'+{improvements[i][2]:.2f}%' if improvements[i][1] >= 0 else f'{improvements[i][2]:.2f}%'
        table_data.append([
            m.capitalize(),
            f'{cnn_vals[i]:.4f}',
            f'{tcn_vals[i]:.4f}',
            improvement_str
        ])
    
    table = axes[1, 1].table(cellText=table_data, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    for i in range(len(table_data)):
        if i == 0:
            for j in range(4):
                table[(i, j)].set_facecolor('#4CAF50')
                table[(i, j)].set_text_props(weight='bold', color='white')
        else:
            if tcn_vals[i-1] > cnn_vals[i-1]:
                table[(i, 3)].set_facecolor('#E8F5E9')
            else:
                table[(i, 3)].set_facecolor('#FFEBEE')
    
    axes[1, 1].set_title('TCN vs CNN Performance Summary', pad=20, fontsize=12, weight='bold')
    
    plt.tight_layout()
    plt.savefig('deepfake_comparison_improved.png', dpi=300, bbox_inches='tight')
    print("\n✓ Results saved to 'deepfake_comparison_improved.png'")
    plt.show()


def main():
    """Main execution function"""
    try:
        # Load dataset with augmentation for training
        print("\n=== Creating training dataset (with augmentation) ===")
        train_dataset_full = VideoDataset(DATA_PATHS, SEQUENCE_LENGTH, IMG_SIZE, 
                                         MAX_VIDEOS_PER_CLASS, augment=True)
        
        print("\n=== Creating validation/test dataset (no augmentation) ===")
        eval_dataset_full = VideoDataset(DATA_PATHS, SEQUENCE_LENGTH, IMG_SIZE, 
                                        MAX_VIDEOS_PER_CLASS, augment=False)
        
        # Split dataset
        total_size = len(train_dataset_full)
        train_size = int(0.7 * total_size)
        val_size = int(0.15 * total_size)
        test_size = total_size - train_size - val_size
        
        # Use augmented data for training
        train_indices = list(range(train_size))
        # Use non-augmented data for validation and testing
        val_indices = list(range(train_size, train_size + val_size))
        test_indices = list(range(train_size + val_size, total_size))
        
        train_dataset = torch.utils.data.Subset(train_dataset_full, train_indices)
        val_dataset = torch.utils.data.Subset(eval_dataset_full, val_indices)
        test_dataset = torch.utils.data.Subset(eval_dataset_full, test_indices)
        
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                                 num_workers=0, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                               num_workers=0, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                                num_workers=0, pin_memory=True)
        
        print(f"\nDataset split: Train={train_size}, Val={val_size}, Test={test_size}")
        
        # Train CNN
        print("\n" + "="*70)
        print("TRAINING IMPROVED CNN MODEL")
        print("="*70)
        cnn_model = ImprovedCNN().to(DEVICE)
        cnn_history = train_model(cnn_model, train_loader, val_loader, "CNN", EPOCHS)
        cnn_results = evaluate_model(cnn_model, test_loader, "CNN")
        
        # Train TCN
        print("\n" + "="*70)
        print("TRAINING IMPROVED TCN MODEL")
        print("="*70)
        tcn_model = ImprovedTCN().to(DEVICE)
        tcn_history = train_model(tcn_model, train_loader, val_loader, "TCN", EPOCHS)
        tcn_results = evaluate_model(tcn_model, test_loader, "TCN")
        
        # Plot results
        plot_results(cnn_history, tcn_history, cnn_results, tcn_results)
        
        print("\n" + "="*70)
        print("EXPERIMENT COMPLETED SUCCESSFULLY!")
        print("="*70)
        print(f"\n📊 FINAL RESULTS:")
        print(f"  CNN - Accuracy: {cnn_results['accuracy']:.4f} | F1-Score: {cnn_results['f1']:.4f}")
        print(f"  TCN - Accuracy: {tcn_results['accuracy']:.4f} | F1-Score: {tcn_results['f1']:.4f}")
        print(f"\n🎯 TCN Improvement over CNN:")
        print(f"  Accuracy: {(tcn_results['accuracy'] - cnn_results['accuracy']):.4f} "
              f"({((tcn_results['accuracy'] - cnn_results['accuracy'])/cnn_results['accuracy']*100):.2f}%)")
        print(f"  F1-Score: {(tcn_results['f1'] - cnn_results['f1']):.4f}")
        
    except Exception as e:
        print(f"\n! CRITICAL ERROR: {e}")
        print("! Check your data paths and ensure videos exist.")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

=== Checking and installing dependencies ===

=== Using device: cpu ===

=== Creating training dataset (with augmentation) ===

=== Loading dataset ===
Loaded 50 real videos
Loaded 10 NeuralTextures videos
Loaded 10 FaceSwap videos
Loaded 10 Deepfakes videos
Loaded 10 Face2Face videos
Loaded 10 FaceShifter videos
Total fake videos: 50
Total dataset size: 100 videos

=== Creating validation/test dataset (no augmentation) ===

=== Loading dataset ===
Loaded 50 real videos
Loaded 10 NeuralTextures videos
Loaded 10 FaceSwap videos
Loaded 10 Deepfakes videos
Loaded 10 Face2Face videos
Loaded 10 FaceShifter videos
Total fake videos: 50
Total dataset size: 100 videos

Dataset split: Train=70, Val=15, Test=15

TRAINING IMPROVED CNN MODEL

Training CNN


Epoch 1/25 [Train]: 100%|██████████| 9/9 [24:25<00:00, 162.85s/it, loss=0.7051]


Epoch 1: Train Loss: 0.7140, Train Acc: 0.4714 | Val Loss: 0.7197, Val Acc: 0.4000
✓ Saved best model with Val Acc: 0.4000


Epoch 2/25 [Train]: 100%|██████████| 9/9 [25:17<00:00, 168.57s/it, loss=0.7744]


Epoch 2: Train Loss: 0.6970, Train Acc: 0.6286 | Val Loss: 0.7382, Val Acc: 0.4000


...

Epoch 12/25 [Train]: 100%|██████████| 9/9 [12:12<00:00, 81.38s/it, loss=0.4911]


Epoch 12: Train Loss: 0.6765, Train Acc: 0.6429 | Val Loss: 0.7513, Val Acc: 0.6000


Epoch 13/25 [Train]: 100%|██████████| 9/9 [11:53<00:00, 79.24s/it, loss=0.3655]


Epoch 13: Train Loss: 0.5978, Train Acc: 0.7286 | Val Loss: 0.7581, Val Acc: 0.6000


Epoch 14/25 [Train]: 100%|██████████| 9/9 [12:45<00:00, 85.11s/it, loss=0.4386]
